In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sympy import *
from robot_tools import *
import math

In [30]:
def velocity_COM(frame, transform_low_high, P, v, omega, alias):
    rotation_high_low = transform_low_high.T[:3, :3]

    symprint('\Omega_G', frame, frame)
    OmegaGee = simplify(rotation_high_low * omega)
    matprint(OmegaGee, alias)

    symprint('V_G', frame, frame)
    VGee = simplify(v + omega.cross(P))
    matprint(VGee, alias)
    
    return OmegaGee, VGee

### 5.7.1 Variables

In [117]:
t = symbols('t')
theta1 = Function(r"\theta_1")(t)
theta1_dot = diff(theta1, t)
theta1_ddot = diff(theta1, t, t)
lg = symbols('l_g')
l = symbols('l')

alias = {}

# display as theta dot instead of d(theta)/dt, and ignore dependency (t)
alias.update({theta1: symbols(r"\theta_1"),
              theta1_dot: symbols(r"\dot{\theta_1}"),
              theta1_ddot: symbols(r"\ddot{\theta_1}"),
             })

In [118]:
# Q1

DH1 = [0, 0, 0, theta1]
T01 = Tlink(DH1)
symprint('T',0, 1)
matprint(T01, alias)

# IMPORTANT: Treat the centre of mass as a separate joint!
DH2 = [lg, 0, 0, 0]
T1G = Tlink(DH2)
symprint('T',1, 'G')
matprint(T1G, alias)

T0G = simplify(T01 * T1G)
symprint('T', 0, 'G')
matprint(T0G, alias)

DHE = [l, 0, 0, 0]
T1E = Tlink(DHE)
symprint('T',1, 'E')
matprint(T1E, alias)

T0E = simplify(T01 * T1E)
symprint('T', 0, 'E')
matprint(T0E, alias)


^0T_1

Matrix([
[cos(\theta_1), -sin(\theta_1), 0, 0],
[sin(\theta_1),  cos(\theta_1), 0, 0],
[            0,              0, 1, 0],
[            0,              0, 0, 1]])

^1T_G

Matrix([
[1, 0, 0, l_g],
[0, 1, 0,   0],
[0, 0, 1,   0],
[0, 0, 0,   1]])

^0T_G

Matrix([
[cos(\theta_1), -sin(\theta_1), 0, l_g*cos(\theta_1)],
[sin(\theta_1),  cos(\theta_1), 0, l_g*sin(\theta_1)],
[            0,              0, 1,                 0],
[            0,              0, 0,                 1]])

^1T_E

Matrix([
[1, 0, 0, l],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

^0T_E

Matrix([
[cos(\theta_1), -sin(\theta_1), 0, l*cos(\theta_1)],
[sin(\theta_1),  cos(\theta_1), 0, l*sin(\theta_1)],
[            0,              0, 1,               0],
[            0,              0, 0,               1]])

In [93]:
# Q2

omega = Matrix([0, 0, 0])
v = Matrix([0, 0, 0])

omega_11, v_11 = revolute_joint(frame = 1, 
                                theta_dot = theta1_dot, 
                                transform_low_high = T01, 
                                omega = omega, v = v, 
                                Display = False,
                                alias = alias)


# IMPORTANT: Treat the centre of mass as a separate joint!
omega_GG, v_GG = revolute_joint(frame = 'e', 
                                theta_dot = theta1_dot, 
                                transform_low_high = T1G, 
                                omega = omega_11, v = v_11, 
                                alias = alias)


Jee, J0 = Jacobian(parameters = [theta1_dot], 
                     v_ee = v_GG, omega_ee = omega_GG, 
                     transform_low_high = T0G, 
                     alias = alias, Display=True, Display_all_details=False)

# Find velocity of G in frame {0}
symprint('V', 0, 'G')
v_0G = simplify(T0G[:3, :3] * v_GG)
matprint(v_0G, alias)

symprint('\\Omega', 0, 'G')
omega_0G = simplify(T0G[:3, :3] * omega_GG)
matprint(omega_0G, alias)


^e\Omega_e

Matrix([
[             0],
[             0],
[\dot{\theta_1}]])

^eV_e

Matrix([
[                 0],
[\dot{\theta_1}*l_g],
[                 0]])

^eJ_e

...

Matrix([
[-l_g*sin(\theta_1)],
[ l_g*cos(\theta_1)],
[                 0],
[                 0],
[                 0],
[                 1]])

^0V_G

Matrix([
[-\dot{\theta_1}*l_g*sin(\theta_1)],
[ \dot{\theta_1}*l_g*cos(\theta_1)],
[                                0]])

^0\Omega_G

Matrix([
[             0],
[             0],
[\dot{\theta_1}]])

In [94]:
# Q3

# Differentiate with respect to t

symprint('\dot{V}', 0, 'G')

A0G = simplify(v_0G.diff(t))
matprint(A0G, alias)

^0\dot{V}_G

Matrix([
[-l_g*(\ddot{\theta_1}*sin(\theta_1) + \dot{\theta_1}**2*cos(\theta_1))],
[ l_g*(\ddot{\theta_1}*cos(\theta_1) - \dot{\theta_1}**2*sin(\theta_1))],
[                                                                     0]])

In [95]:
# Q4

# Find velocity of G in frame {1}

symprint('V', 1, 'G')
v_1G = simplify(T1G[:3, :3] * v_GG)
matprint(v_1G, alias)

symprint('\\Omega', 0, 'G')
omega_1G = simplify(T1G[:3, :3] * omega_GG)
matprint(omega_1G, alias)

#------------------------------

# Find acceleration

# DON'T DIRECTLY APPLY ELEMENT-WISE DIFFERENTIATE, EXCEPT WITH RESPECT TO {0}
# Differentiate with respect to t
# symprint('\dot{V}', 1, 'G')
# A1G = simplify(v_1G.diff(t))
# matprint(A1G, alias)

# Find A1G by transforming from A0G
symprint('\dot{V}', 1, 'G')
A1G = simplify(T01[:3, :3]**-1 * A0G)
matprint(A1G, alias)

^1V_G

Matrix([
[                 0],
[\dot{\theta_1}*l_g],
[                 0]])

^0\Omega_G

Matrix([
[             0],
[             0],
[\dot{\theta_1}]])

^1\dot{V}_G

Matrix([
[-\dot{\theta_1}**2*l_g],
[   \ddot{\theta_1}*l_g],
[                     0]])

In [109]:
# Q5

def angular_acce(frame, joint_type, transform_low_high, omega_dot, theta_dot, theta_ddot, alias, Display = True):
    # Transpose and extract the 3x3 matrix
    rotation_high_low = transform_low_high.T[:3, :3]
    
    if joint_type == 'r':
        omega_new = simplify(rotation_high_low * omega_dot +
                             rotation_high_low * omega_dot.cross(theta_dot * Matrix([0, 0, 1])) +
                             theta_ddot * Matrix([0, 0, 1])
                            )
        if Display == True:
            symprint('\dot{\\Omega}', frame, frame)
            matprint(omega_new, alias)
            
    elif joint_type == 'p':
        omega_new = simplify(rotation_high_low * omega_dot)
        
        if Display == True:
            symprint('\dot{\\Omega}', frame, frame)
            matprint(omega_new, alias)
            
    else:
        print('Joint type not found, try r and p')
        return None
    
    
    return omega_new


def linear_acce(frame, joint_type, transform_low_high, omega, omega_dot, v_dot, alias, Display = True):
    
    # Transpose and extract the 3x3 matrix
    rotation_high_low = transform_low_high.T[:3, :3]
    P = transform_low_high[:3, -1]
    
    if joint_type == 'r':

        v_new = simplify(rotation_high_low * (v_dot + omega_dot.cross(P) + omega.cross(omega.cross(P))))
        
        if Display == True:
            symprint('\dot{V}', frame, frame)
            matprint(v_new, alias)
    elif joint_type == 'p':
        pass
        
    else:
        print('Joint type not found, try r and p')
        return None
    
    return v_new

def force_cal(mass, v_dot):
    F = simplify(mass * v_dot)  
    return F
    
def moment_cal(I, omega_dot, omega):
    
    intermediate = I * omega
    M = simplify(I * omega_dot + omega.cross(intermediate))
    return M

    
    

In [111]:
omega_00 = Matrix([0, 0, 0])
omega_dot_00 = Matrix([0, 0, 0])
v_00 = Matrix([0, 0, 0])
g = symbols('g')
v_dot_00 = Matrix([0, 0, g])


omega_11, v_11 = revolute_joint(frame = 1, 
                                theta_dot = theta1_dot, 
                                transform_low_high = T01, 
                                omega = omega, v = v, 
                                Display = False,
                                alias = alias)

omega_dot_11 = angular_acce(frame = 1,
                 joint_type = 'r',
                 transform_low_high = T01,
                 omega_dot = omega_dot_00,
                 theta_dot = theta1_dot,
                 theta_ddot = theta1_ddot,
                 alias = alias)

v_dot_11 = linear_acce(frame = 1,
                 joint_type = 'r',
                 transform_low_high = T01,
                 omega = omega_00,
                 omega_dot = omega_dot_00,
                 v_dot = v_dot_00,
                 alias = alias)

v_dot_GG = linear_acce(frame = 'G',
                 joint_type = 'r',
                 transform_low_high = T1G,
                 omega = omega_11,
                 omega_dot = omega_dot_11,
                 v_dot = v_dot_11,
                 alias = alias)

# Calculate force
Ixx = symbols('I_{xx}')
Iyy = symbols('I_{yy}')
Izz = symbols('I_{zz}')

I = Matrix([[Ixx, 0, 0],
            [0, Iyy, 0],
            [0, 0, Izz]])

m1 = symbols('m_1')

F1G = force_cal(mass = m1, 
              v_dot = v_dot_GG)
symprint('F', 1, 'G')
matprint(F1G, alias)

M1G = moment_cal(I = I, 
               omega_dot = omega_dot_11, 
               omega = omega_11)
symprint('M', 1, 'G')
matprint(M1G, alias)


^1\dot{\Omega}_1

Matrix([
[              0],
[              0],
[\ddot{\theta_1}]])

^1\dot{V}_1

Matrix([
[0],
[0],
[g]])

^G\dot{V}_G

Matrix([
[-\dot{\theta_1}**2*l_g],
[   \ddot{\theta_1}*l_g],
[                     g]])

^1F_G

Matrix([
[-\dot{\theta_1}**2*l_g*m_1],
[   \ddot{\theta_1}*l_g*m_1],
[                     g*m_1]])

^1M_G

Matrix([
[                     0],
[                     0],
[I_{zz}*\ddot{\theta_1}]])

In [112]:
def inward_iteration(transform_low_high, Fnext, Fcom):
 
    rotation_low_high = transform_low_high[:3, :3]
    
    Fprev = simplify(rotation_low_high * Fnext + Fcom)

    Mprev = Mcom + rotation_low_high * Mnext + Pcom.cross(Fcom) + P.cross(rotation_low_high * Fnext)
    
    return Fprev








In [116]:
Fee = Matrix([0, 0, 0])
Mee = Matrix([0, 0, 0])
F11 = inward_iteration(transform_low_high = T1G, 
                 Fnext = Fee, 
                 Fcom = F1G)
matprint(F11, alias)

Matrix([
[-\dot{\theta_1}**2*l_g*m_1],
[   \ddot{\theta_1}*l_g*m_1],
[                     g*m_1]])

### 5.7.2 Variables

In [2]:
t = symbols('t')
theta1 = Function(r"\theta_1")(t)
theta1_dot = diff(theta1, t)
theta1_ddot = diff(theta1, t, t)

theta2 = Function(r"\theta_2")(t)
theta2_dot = diff(theta2, t)
theta2_ddot = diff(theta2, t, t)

alias = {}

# display as theta dot instead of d(theta)/dt, and ignore dependency (t)
alias.update({theta1: symbols(r"\theta_1"),
              theta1_dot: symbols(r"\dot{\theta_1}"),
              theta1_ddot: symbols(r"\ddot{\theta_1}"),
              theta2: symbols(r"\theta_2"),
              theta2_dot: symbols(r"\dot{\theta_2}"),
              theta2_ddot: symbols(r"\ddot{\theta_2}")
             })

In [3]:
# Q1

L1 = symbols('L1')
L2 = symbols('L2')
theta1 = symbols('\\theta_1')
theta2 = symbols('\\theta_2')

DH1 = [0, 0, 0, theta1]
T01 = Tlink(DH1)
symprint('T',0, 1)
matprint(T01)

DH2 = [L1, -pi/2, 0, theta2]
T12 = Tlink(DH2)
symprint('T',1, 2)
matprint(T12)

T02 = simplify(T01 * T12)
symprint('T',0, 2)
matprint(T02)


^0T_1

Matrix([
[cos(\theta_1), -sin(\theta_1), 0, 0],
[sin(\theta_1),  cos(\theta_1), 0, 0],
[            0,              0, 1, 0],
[            0,              0, 0, 1]])

^1T_2

Matrix([
[ cos(\theta_2), -sin(\theta_2), 0, L1],
[             0,              0, 1,  0],
[-sin(\theta_2), -cos(\theta_2), 0,  0],
[             0,              0, 0,  1]])

^0T_2

Matrix([
[cos(\theta_1)*cos(\theta_2), -sin(\theta_2)*cos(\theta_1), -sin(\theta_1), L1*cos(\theta_1)],
[sin(\theta_1)*cos(\theta_2), -sin(\theta_1)*sin(\theta_2),  cos(\theta_1), L1*sin(\theta_1)],
[             -sin(\theta_2),               -cos(\theta_2),              0,                0],
[                          0,                            0,              0,                1]])

In [13]:
# Q2 

omega = Matrix([0, 0, 0])
v = Matrix([0, 0, 0])

# theta_1_dot = symbols('\dot{\\theta_1}')
# theta_2_dot = symbols('\dot{\\theta_2}')

omega_11, v_11 = revolute_joint(frame = 1, 
                                theta_dot = theta1_dot, 
                                transform_low_high = T02, 
                                omega = omega, v = v, 
                                alias = alias)

omega_22, v_22 = revolute_joint(frame = 2, 
                                theta_dot = theta2_dot, 
                                transform_low_high = T02, 
                                omega = omega_11, v = v_11, 
                                alias = alias)


# Location of the centre of mass
lg2 = symbols('Lg_2')
PG22 = Matrix([lg2, 0, 0])


VG22 = velocity_COM(2, 
                    P = PG22,
                    v = v_22,
                    omega = omega_22, 
                    alias = alias)

symprint('V_G', 0, 2)

# Velocity of G2 in {0} = rotational matrix from 0 to 2 * Velocity of G2 in {2}
VG02 = simplify(T02[:3, :3] * VG22)
matprint(VG02, alias)

^1\Omega_1

Matrix([
[             0],
[             0],
[\dot{\theta_1}]])

^1V_1

Matrix([
[0],
[0],
[0]])

^2\Omega_2

...

Matrix([
[                0],
[                0],
[L1*\dot{\theta_1}]])

^2V_G_2

Matrix([
[                                       0],
[                     Lg_2*\dot{\theta_2}],
[\dot{\theta_1}*(L1 + Lg_2*cos(\theta_2))]])

^0V_G_2

Matrix([
[-Lg_2*\dot{\theta_2}*sin(\theta_2)*cos(\theta_1) - \dot{\theta_1}*(L1 + Lg_2*cos(\theta_2))*sin(\theta_1)],
[-Lg_2*\dot{\theta_2}*sin(\theta_1)*sin(\theta_2) + \dot{\theta_1}*(L1 + Lg_2*cos(\theta_2))*cos(\theta_1)],
[                                                                       -Lg_2*\dot{\theta_2}*cos(\theta_2)]])

In [14]:
# Q3
matprint(VG02.diff(t), alias)

Matrix([
[-Lg_2*\ddot{\theta_2}*sin(\theta_2)*cos(\theta_1) - \ddot{\theta_1}*(L1 + Lg_2*cos(\theta_2))*sin(\theta_1)],
[-Lg_2*\ddot{\theta_2}*sin(\theta_1)*sin(\theta_2) + \ddot{\theta_1}*(L1 + Lg_2*cos(\theta_2))*cos(\theta_1)],
[                                                                        -Lg_2*\ddot{\theta_2}*cos(\theta_2)]])